In [2]:
from random import uniform, random
from math import e, sqrt,cos,pi
import numpy as np

In [13]:
class Bat():
    def __init__(self,frecuency_min, frecuency_max, position, velocity) -> None:
        self.frecuency_min = frecuency_min
        self.frecuency_max = frecuency_max
        self.current_frecuency = uniform(frecuency_min,frecuency_max)
        self.loudness = 1
        random_number = random()
        self.pulse_interval_initial = random_number
        self.current_pulse_interval = random_number
        self.velocity = np.array(velocity)
        self.position = np.array(position)

    def update_frecuency(self):
        beta = random()
        self.current_frecuency = self.frecuency_min + (self.frecuency_max-self.frecuency_min)*beta

    def update_velocity(self, best_sol):
        self.velocity = self.velocity + (self.position - best_sol)*self.current_frecuency
    
    def update_position(self):
        self.position = self.position + self.velocity
    
    def update_loudness(self, alfa):
        self.loudness = alfa* self.loudness

    def update_pulse_interval(self, gamma, t):
        self.current_pulse_interval = self.pulse_interval_initial*(1-(e**(gamma*-1*t)))

    def fly_randomly(self, average_loudness, flying_initial):
        epsilon = random()
        self.position = flying_initial+epsilon*average_loudness
    


class BA():
    def __init__(self, number_of_bats, num_dimentions, interval, alfa, gamma, number_of_iterations=50) -> None:
        self.bats = list()
        self.number_of_iterations = number_of_iterations
        self.alfa = alfa
        self.gamma = gamma
        for _ in range(0,number_of_bats):
            ##todo check velocity and init position
            self.bats.append(Bat(frecuency_min=0, frecuency_max=100,
             position=[uniform(interval[0], interval[1]) for _ in range(0,num_dimentions)], velocity=[uniform(interval[0], interval[1]) for _ in range(0,num_dimentions)]))

    def get_best_position(self,function):
        best_bat = self.bats[0]
        current_cost_best = function(best_bat.position)
        for bat in self.bats:
            current_cost_bat = function(bat.position)
            if(current_cost_bat < current_cost_best):
                best_bat = bat
                current_cost_best = current_cost_bat
        return best_bat.position, current_cost_best
    
    def get_average_loudness(self):
        average = 0
        for bat in self.bats:
            average += bat.loudness
        return average/len(self.bats)
    
    def run(self, function):
        solution_position = self.get_best_position(function)
        for t in range(1, self.number_of_iterations):
            best_position, best_cost = self.get_best_position(function)
            average_loudness = self.get_average_loudness()
            for bat in self.bats:
                random_number = random()
                bat.update_frecuency()
                bat.update_velocity(best_position)
                bat.update_position()
                if(random_number > bat.current_pulse_interval):
                    bat.fly_randomly(average_loudness, best_position)
                bat.fly_randomly(average_loudness, bat.position)
                if(random_number < bat.loudness and function(bat.position) < best_cost):
                    #todo accept solutions
                    solution_position = best_position
                    bat.update_loudness(self.alfa)
                    bat.update_pulse_interval(self.gamma, t)
        return solution_position


## TEST'S

### Himmelblau's function

![alt text](him.png "Title")

In [4]:
fx = lambda x : (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 -7)**2

In [15]:
ba = BA(number_of_bats=20, num_dimentions=2, interval=[-5,5], number_of_iterations=50, alfa= 0.9, gamma=0.9)
ba.run(fx)

/tmp/ipykernel_16330/3189032922.py:1: RuntimeWarning: overflow encountered in double_scalars
  fx = lambda x : (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 -7)**2


(array([ 3.53702562, -1.62861243]), 0.6710044209045315)

### Ackley function

![alt text](akley.png "Title")

In [16]:
fx = lambda x : -20*e**(-0.2*sqrt(0.5*(x[0]**2 + x[1]**2)))-e**(0.5*(cos(2*pi*x[1])+cos(2*pi*x[0])))+e+20

In [17]:
ba = BA(number_of_bats=20, num_dimentions=2, interval=[-5,5], number_of_iterations=50, alfa= 0.9, gamma=0.9)
ba.run(fx)

array([-4.05596232e+39, -2.96126147e+39])

### Rastrigin

![alt text](ras.png "Title")

In [18]:
fx = lambda x : 10*2 + (x[0]**2 - 10*cos(2*pi*x[0])) + (x[1]**2 - 10*cos(2*pi*x[1]))

In [32]:
ba = BA(number_of_bats=20, num_dimentions=2, interval=[-5.12,5.12], number_of_iterations=50, alfa= 0.9, gamma=0.9)
ba.run(fx)

array([1.70775759, 1.01738101])